# Triangle model
This interactive notebook runs a triangle model


## Run parameters 
This block is necessary for running with [papermill](https://papermill.readthedocs.io/en/latest/)

In [1]:
code_name = "_boo2"
batch_name = None

# Model configs
ort_units = 119
pho_units = 250
sem_units = 2446
hidden_os_units = 500
hidden_op_units = 150
hidden_ps_units = 500
hidden_sp_units = 500
pho_cleanup_units = 50
sem_cleanup_units = 50
pho_noise_level = 0.0
sem_noise_level = 0.0
activation = "sigmoid"

tau = 1 / 3
max_unit_time = 4.0
output_ticks = 13
inject_error_ticks = 8

# Training configs
pretrain_checkpoint = None
optimizer = "adam"
learning_rate = 0.05
zero_error_radius = 0.1
save_freq = 10

# Environment configs
wf_compression = "log"
wf_clip_low = 0
wf_clip_high = 10_000

task_names = ['ort_pho', 'pho_pho', 'triangle']
tasks_ps = [0.2, 0.3, 0.5]

total_sample = 1_000_000
batch_size = 200
rng_seed = 2021
which_gpu = 0


## System environment

In [2]:
import meta
meta.split_gpu(which_gpu=which_gpu)  # IMPORTANT: do not import TensorFlow before this line

import os
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Set all seeds
os.environ["PYTHONHASHSEED"] = str(rng_seed)
tf.random.set_seed(rng_seed)
np.random.seed(rng_seed)

# Loads .env file
load_dotenv()  


True

## Create run configuration
`Config()` stores all the run setting in a class. `**globals()` is used to access all global variables in the parameter block.

In [3]:
# cfg = meta.Config.from_json(os.path.join(tf_root, "models", batch_name, code_name, "model_config.json"))   # Load from json
cfg = meta.Config.from_dict(**globals())
print(cfg)

UUID not found, regenerating.
Saved config json to /home/jal21012/triangle_model/models/_boo2/model_config.json
Config(code_name='_boo2', model_config=ModelConfig(ort_units=119, pho_units=250, sem_units=2446, hidden_os_units=500, hidden_op_units=150, hidden_ps_units=500, hidden_sp_units=500, pho_cleanup_units=50, sem_cleanup_units=50, pho_noise_level=0.0, sem_noise_level=0.0, activation='sigmoid', tau=0.3333333333333333, max_unit_time=4.0, output_ticks=13, inject_error_ticks=8, pretrain_checkpoint=None, optimizer='adam', learning_rate=0.05, zero_error_radius=0.1), environment_config=EnvironmentConfig(task_names=['ort_pho', 'pho_pho', 'triangle'], wf_compression='log', wf_clip_low=0, wf_clip_high=10000, total_sample=1000000, batch_size=200, tasks_ps=[0.2, 0.3, 0.5]), rng_seed=2021, save_freq=10, uuid='a15b09e6bdd540fca570e528183e85a9', batch_name=None, batch_unique_setting_string=None)


## Create Experience
- `Experience()` defines what the model is trained on. It consists of one or more `Stage()`. 
- Each `Stage()` describes what tasks are the model trained with, and how often a task is used during training. It contains one or more `Task()`. 
- Each `Task()` contains how fast the corpus is opened (a set of word that can be sampled), defaults to full open.
- See the docstrings in each object for further details.
- CAUTION: if modified to multiple `Stage()`, Config() may not be storing everything needed to recreate Experience(). 

In [4]:
from environment import Task, Stage, Experience

stages = [
    Stage(
        name="one",
        tasks=[Task(x) for x in cfg.task_names],
        stage_sample=cfg.total_sample,
        task_probability_start=cfg.tasks_ps,
    )
]

experience = Experience(stages)

## Create model trainer
- It includes model, optimizer, metrics, and loss function
- Since each sub-task has its own states, it must be trained with separate optimizer.


In [5]:
from training import Trainer, TensorBoardManager
trainer = Trainer(cfg, experience)

## Train model
try_to_resume wiil restore from latest checkpoint if it exists. However, Environment() will no longer be identical if resume. 

In [6]:
tb_manager = TensorBoardManager(cfg, trainer.model, trainer.train_metrics, trainer.train_losses)
trainer.train(tensorboard_manager=tb_manager, try_to_resume=True)

Training:   0%|          | 0/100 [00:00<?, ?it/s]

Initializing from scratch.


Training: 100%|██████████| 100/100 [02:16<00:00,  1.20s/it, Triangle PHO ACC: 0.65 | Triangle SEM ACC: 0.00 | ort_pho ACC: 0.70 | pho_pho ACC: 0.67]

## Run tests

In [7]:
import benchmarks
# Basic test
# benchmarks.run_test1(cfg.code_name, cfg.batch_name)  # Basic accuracy test only
# benchmarks.run_test1s(cfg.code_name, cfg.batch_name)  # Basic accuracy test only

## All benchmarks
# benchmark.main(cfg.code_name, cfg.batch_name)  


## Full training set test
# import evaluate
# test = evaluate.Test(cfg)
# test.eval_train("triangle", to_bq=True)
